# MAVIS Astrometric Simulator

## Phase A Version
March 2020

## Todo
1. Update the PSDdatabase

# Imports

In [1]:
# Standard
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Astropy
from astropy.io import fits, ascii
from astropy.table import Table

# Project-specific/
#sys.path.append('/src')
cwd = os.getcwd()
os.chdir(os.getcwd() + "/src")

#import mavisim.rampup
#import mavisim.addconstantsky
#import mavissimim.addnoise
#import mavissimim.trimimage
import mavisim.input_parameters as input_par
#from rebin import rebin

from mavisim.Source import Source
#from mavissimim.AOGaussGrid import AOGaussGrid
#from mavissimim.SeeingGrid import SeeingGrid
from mavisim.InputCoo import InputCoo

os.chdir(cwd)


# Load the input data

In [2]:
# Nbody input plus chosen exposure time
ngc3201 = input_par.nbody_in

exp_time = 3600

# Create the source object (to populate the image)

In [3]:
source = Source(ngc3201, exp_time, tt_amp_fac=1.0, static_dist=False, stat_amp=1.0).main()
#self, mavis_src, exp_time, static_dist, tt_amp_fac, stat_amp

No TT kernel applied
No static distortion applied.


In [ ]:
print (source[0].colnames)

# Create the input catalogue to compare with DAOPhot

In [4]:
input_coo = InputCoo(source).main()

# Create the convolved AO core grid and the Gaussian grid

## Jesse, this is where you would pick it up

In [ ]:
ao_field, gauss_field = AOGaussGrid(source, fv_psf=False).main()

# Use the large PSF to create a grid of seeing wings

In [ ]:
seeing_field = SeeingGrid(gauss_field).main()

# Make the final image

In [ ]:
image = ao_field + seeing_field

# Add sky background

In [ ]:
#image_plussky, sky_val = mavissimim.addconstantsky.add_constant_sky(image, exp_time)

# Add all noise

**Idea** = randomly generate a read_noise frame to simulate a bias image

In [ ]:
#image_plussky_rn = mavissimim.addnoise.add_read_noise(image_plussky, sky_val)
# Add shot noise, read noise and convert from electrons to ADU
image_allnoise = mavissimim.addnoise.add_all_noise(image, source.meta["exp_time"])

# Trim the final image and the input catalogue

In [ ]:
(image_plusskyrn_trim, trimmed_input_coo) = mavissimim.trimimage.trim_image(image_allnoise, input_coo)
#(image_plusskyrn_trim, trimmed_input_coo) = mavissimim.trimimage.trim_image(image_plussky_rn, input_coo)


# Save the final image and input catalogue

In [ ]:
final_image = np.array(image_plusskyrn_trim, dtype="float32")
hdu = fits.PrimaryHDU(final_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + ".fits", overwrite=True)

In [ ]:
ao_image = np.array(ao_frame_trim, dtype="float32")
hdu = fits.PrimaryHDU(ao_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'AOField.fits', overwrite=True)

In [ ]:
gauss_image = np.array(gauss_frame_trim, dtype="float32")
hdu = fits.PrimaryHDU(gauss_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'GaussField.fits', overwrite=True)

In [ ]:
seeing_image = np.array(sky_plusskyrn_trim, dtype="float32")
hdu = fits.PrimaryHDU(seeing_image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_save_path + file_name + 'SeeField.fits', overwrite=True)

In [ ]:
ascii.write(trimmed_input_coo, coo_save_path + file_name + '.all', overwrite=True)

In [ ]:
print (trimmed_input_coo["CCD_Mapped_X"][0])
print (trimmed_input_coo["CCD_Mapped_Y"][0])
print (trimmed_input_coo["Flux"][0])